In [31]:
#this number can be played with for new results if we have time
SMOOTHING_FACTOR = 0.5

class Dataset:
    def __init__(self, images_file, labels_file, lines_per_image = 28):
        self.images = []
        self.labels = []
        self.images_by_label = {}
        self.priors = {}
        with open(images_file) as file:
            EOF = False
            while not EOF:
                image = []
                for i in range(lines_per_image):
                    line = file.readline()
                    if not line:
                        EOF = True
                        break
                    image.append(list(line))
                if EOF:
                    break
                self.images.append(image)
        with open(labels_file) as file:
            for line in file:
                self.labels.append(line)
class Classifier:
    def __init__(self, images, label):
        self.images = images
        self.label = label
        
    def evaluate_likelihood(self, image):
        return 0
        
class HeatMap:
    def __init__(self, label, mapSize = 28):
        self.hMap = [[0 for i in range(mapSize)] for j in range(mapSize)]
        self.label = label
        self.mapSize = mapSize
        self.totalExamples = 0
                    
    def printHMap(self):
        print("Heat map for digit: " + str(self.label) + "from " + str(self.totalExamples) + " examples")
        for i in range(self.mapSize):
            print(self.hMap[i])

    def addToHMap(self, image):
        self.totalExamples += 1
        for i in range(self.mapSize):
            for j in range(self.mapSize):
                if(image[i][j] != ' '):
                    self.hMap[i][j] += 1
    
    def smoothHMap(self):
        denominator = SMOOTHING_FACTOR * self.totalExamples
        for i in range(self.mapSize):
            for j in range(self.mapSize):
                numerator = self.hMap[i][j] + SMOOTHING_FACTOR
                self.hMap[i][j] = numerator / denominator
                
    def labelProbability(self, trainingSetSize = 5000):
        return self.totalExamples / trainingSetSize

            
        

In [32]:
dataset = Dataset("trainingimages","traininglabels")

In [33]:
print(dataset.images[0][7][13])
print(len(dataset.images))
print(dataset.labels[0])

#
5000
5



In [34]:
digitHMaps = [None]*10
for i in range(0,10):
    digitHMaps[i] = HeatMap(i)

for i  in range(len(dataset.images)):
    currLabel = int(dataset.labels[i])
    digitHMaps[currLabel].addToHMap(dataset.images[i])
    
for i in range(0,10):
    digitHMaps[i].smoothHMap()
    #print(str(100* digitHMaps[i].labelProbability(len(dataset.images))) + '%')

